In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
from prog import scheduler as sch

In [ ]:
# save current working directory as homey
# homey = os.path.abspath(os.path.dirname(__file__))
homey = os.getcwd() # works in jupyter notebook

In [ ]:
# set directory paths
dataPath = os.path.join(homey, 'data')
progPath = os.path.join(homey, 'prog')
simPath = os.path.join(homey, 'FB_Sim')
forcPath = os.path.join(simPath, 'ForecastRedoux')
# this should probably be moved out of the simulator and into the main sql area, where that ends up
sqlPath = os.path.join(forcPath, 'SQL')

In [ ]:
# set paths to excel files
forecastFilename = os.path.join(dataPath, 'RegularForecast.xlsx')
mfgCentersFilename = os.path.join(dataPath, 'MfgCenters.xlsx')
moFilename = os.path.join(dataPath, 'MOs.xlsx')
laborAvailFilename = os.path.join(dataPath, 'LaborAvailablePerDay.xlsx')
leadFilename = os.path.join(dataPath, 'LeadTimes.xlsx')

In [ ]:
sys.path.insert(0, forcPath)

In [ ]:
import ForecastMain as fm
import ForecastAPI as fa

In [ ]:
# pull the usual FB_Sim queries
# fa.run_queries(queryPath=sqlPath, dataPath=dataPath)

In [ ]:
# save mfgCenters as df, includes MFG Center assignments and Setup/labor time estimates
mfgCenters = pd.read_excel(mfgCentersFilename, header=0)

In [ ]:
# save current Manufacture Orders
modf = pd.read_excel(moFilename, header=0)

In [ ]:
# save lead time estimates
leadTimes = pd.read_excel(leadFilename, header=0)

In [ ]:
# make a date list with labor availability
dateList = sch.create_date_list(dailyLabor=11)

In [ ]:
# run the auto schedule to get an ideal schedule by priority
moLinesLabor = sch.run_auto_schedule(modf=modf, mfgCenters=mfgCenters, dateList=dateList)

In [ ]:
# use the last scheduled FG in an order to save an ideal schedule
idealSchedule = moLinesLabor.drop_duplicates('ORDER', keep='last')

In [ ]:
# replace the schedule dates on the MO order lines with the new dates for those orders
newMOdf = pd.merge(modf.copy(), idealSchedule[['ORDER', 'NewDate']].copy(), how='left', on='ORDER')
newMOdf['DATESCHEDULED'] = newMOdf['NewDate'].copy()
newMOdf.drop(labels='NewDate', axis=1, inplace=True)

In [ ]:
# run the new MO schedule through the FB_Sim to find phantom orders
orderTimeline = fm.run_normal_forecast_tiers_v3(dataPath=dataPath, includeSO=False, subMO=newMOdf.copy())

In [ ]:
# get a list of phantom orders and their grandparent orders
### Currently phantom orders are placed a day before the order with the shortage will finish.  So
###     if the lead time field is used later, this could throw the phantom schedule dates.  I'll
###     try to avoid this by getting back to the shortage date, but it might become redundant later.
phantoms = orderTimeline[orderTimeline['ITEM'] == 'Phantom'].copy()
buyPhantoms = phantoms[phantoms['Make/Buy'] == 'Buy'].copy()


In [ ]:
# add the lead times to each part
leadPhantoms = pd.merge(buyPhantoms.copy(), leadTimes[['PART','LeadTimes']].copy(), how='left', on='PART')

In [ ]:
# simplify the dataFrame - not necessary
orderLeads = leadPhantoms[['GRANDPARENT','LeadTimes']].copy()

In [ ]:
# sort by grandparent and then lead time to make sure the highest lead time comes first
orderLeads.sort_values(by=['GRANDPARENT','LeadTimes'], ascending=[True, False], inplace=True)

In [ ]:
# drop duplicate grandparents leaving the longest lead as the only one remaining
orderLeads.drop_duplicates('GRANDPARENT', keep='first', inplace=True)

In [ ]:
# rename column for easy merge
orderLeads.rename(columns={'GRANDPARENT':'ORDER'}, inplace=True)

In [ ]:
# save a new copy of the modf with longest leads added
leadMOdf = pd.merge(modf.copy(), orderLeads.copy(), how='left', on='ORDER')

In [ ]:
# fill missing leads with 0 since that is effectively what is required
leadMOdf.fillna(0, inplace=True)

In [ ]:
modf

In [ ]:
leadMOdf

In [ ]:
leadMOdfSave = leadMOdf.copy()
mfgCentersSave = mfgCenters.copy()
dateListSave = dateList.copy()

In [ ]:
# getting a list of Finished Goods on MO's
moFgOnly = leadMOdf[leadMOdf['ORDERTYPE'] == 'Finished Good'].copy()
# sorting by date so the earliest scheduled can be the highest priority
moFgOnly.sort_values('DATESCHEDULED', inplace=True)
# renaming part column to match MO header
mfgCenters.rename(columns={'Part':'PART'}, inplace=True)
# adding centers and labor estimates to MO lines
moLinesLabor = pd.merge(moFgOnly.copy(), mfgCenters.copy(), how='left', on='PART')
# save missing info for later.  Will want user to see what items were missed for lack of data.
missingCenters = moLinesLabor[moLinesLabor['Mfg Center'].isnull()].copy()
missingSetup = moLinesLabor[moLinesLabor['Setup'].isnull()].copy()
missingLabor = moLinesLabor[moLinesLabor['LaborPer'].isnull()].copy()
# replace nulls with 0 for maths.  Probably not necessary, didn't test.
moLinesLabor.fillna(0, inplace=True)
# create a column for the total labor required for each order
moLinesLabor['LaborRequired'] = moLinesLabor['Setup'] + (moLinesLabor['LaborPer'] * moLinesLabor['QTYREMAINING'])
# calculate cumulative labor needed for builds in their current date order
moLinesLabor['CumulativeLaborRequired'] = np.nan
x = 0
for index in moLinesLabor.index:
    moLinesLabor.at[index, 'CumulativeLaborRequired'] = moLinesLabor.at[index, 'LaborRequired'].copy() + x
    x = moLinesLabor.at[index, 'CumulativeLaborRequired'].copy()

In [ ]:
# just temporary
moLinesLaborSave = moLinesLabor.copy()

In [ ]:
# refresh!!
moLinesLabor = moLinesLaborSave.copy()

In [ ]:
moLinesLabor['NewDate'] = np.nan
for index in moLinesLabor.index:
    laborNeeded = moLinesLabor.at[index, 'CumulativeLaborRequired'].copy()
    tempDateList = dateList[dateList['AvailableLabor'] >= laborNeeded].copy()
    newDate = tempDateList['StartDate'].iat[0]
    moLinesLabor.at[index, 'NewDate'] = newDate

In [ ]:
moLinesLabor['NewDate'] = np.nan
for 

In [ ]:
moLinesLabor = sch.run_auto_schedule(modf=leadMOdf, mfgCenters=mfgCenters, dateList=dateList)

In [ ]:
moLinesLabor

In [ ]:
leadMOdf

In [ ]:
modf

In [ ]:
dateList